In [1]:
import numpy as np
from database.connect import conn

# Real Baby Example

In [2]:
## Real Baby Example
rb_classes = np.load('predictions_realbb.npy')

In [3]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM Baby_Bath_Skin_and_Grooming;") 
rows = cursor.fetchall()
columns = [col[0] for col in cursor.description]

In [4]:
label = 0
indexes_accept_rb = [i for i in range(len(rb_classes)) if rb_classes[i] == label]
y_pred_rb = [1 if i in indexes_accept_rb else 0 for i in range(len(rows))]
indexes_accept_rb.pop(9)
y_true_rb = np.load('ground_truth/label_real_baby.npy')


## Blue Dispenser Pump

In [5]:
blue = np.load('predictions_blue.npy')
y_true_blue = np.load('ground_truth/label_blue_dispenser_head.npy')

In [6]:
np.unique(blue, return_counts=True)

(array([0, 1, 2]), array([  5,  34, 263]))

In [7]:
label = 0
indexes_accept_blue = [i for i in range(len(blue)) if blue[i] == label]
y_pred_blue = [1 if i in indexes_accept_blue else 0 for i in range(len(rows))]

## Dispenser Pump

In [8]:
dh_classes = np.load('predictions_dh.npy')
y_true_dh = np.load('ground_truth/label_dispenser_pump.npy')

In [9]:
label = 0
indexes_accept_dh = [i for i in range(len(dh_classes)) if dh_classes[i] == label]
y_pred_dh = [1 if i in indexes_accept_dh else 0 for i in range(len(rows))]

## Impossible Query Benchmark

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

accuracy = accuracy_score(y_true_rb, y_pred_rb)
print(accuracy)

confusion = confusion_matrix(y_true_rb, y_pred_rb)
print(confusion)

report = classification_report(y_true_rb, y_pred_rb)
print(report)

0.9966887417218543
[[268   1]
 [  0  33]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       269
           1       0.97      1.00      0.99        33

    accuracy                           1.00       302
   macro avg       0.99      1.00      0.99       302
weighted avg       1.00      1.00      1.00       302



In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

accuracy = accuracy_score(y_true_blue, y_pred_blue)
print(accuracy)

confusion = confusion_matrix(y_true_blue, y_pred_blue)
print(confusion)

report = classification_report(y_true_blue, y_pred_blue)
print(report)

1.0
[[297   0]
 [  0   5]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       297
           1       1.00      1.00      1.00         5

    accuracy                           1.00       302
   macro avg       1.00      1.00      1.00       302
weighted avg       1.00      1.00      1.00       302



In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

accuracy = accuracy_score(y_true_dh, y_pred_dh)
print(accuracy)

confusion = confusion_matrix(y_true_dh, y_pred_dh)
print(confusion)

report = classification_report(y_true_dh, y_pred_dh)
print(report)

1.0
[[290   0]
 [  0  12]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       290
           1       1.00      1.00      1.00        12

    accuracy                           1.00       302
   macro avg       1.00      1.00      1.00       302
weighted avg       1.00      1.00      1.00       302



## CLIP Bechmark

In [13]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO
from tqdm import tqdm
import torch

print("Loading CLIP model...")

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
print("Loaded CLIP model")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
print("Loaded CLIP processor")

y_pred_clip = []
print("Evaluating CLIP model on validation set...")
for row in tqdm(rows):
    image_url = row[4]
    # Define your text prompt and load the product image
    text_prompt = "dispenser pump"
    image = Image.open(BytesIO(requests.get(image_url).content))
    # Prepare the inputs
    inputs = processor(text=[text_prompt], images=image, return_tensors="pt", padding=True)

    # Obtain the image and text embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        image_embeddings = outputs.image_embeds  # shape: [batch_size, embedding_dim]
        text_embeddings = outputs.text_embeds    # shape: [batch_size, embedding_dim]

    # Normalize embeddings
    image_embeddings = image_embeddings / image_embeddings.norm(p=2, dim=-1, keepdim=True)
    text_embeddings = text_embeddings / text_embeddings.norm(p=2, dim=-1, keepdim=True)

    # Compute cosine similarity
    cosine_similarity = (image_embeddings * text_embeddings).sum(dim=-1)
    # print("Cosine Similarity:", cosine_similarity.item())

    # Set an empirical threshold (e.g., 0.3 or 0.4) based on validation
    threshold = 0.2
    if cosine_similarity.item() > threshold:
        y_pred_clip.append(1)
    else:
        y_pred_clip.append(0)


/home/nightstalker/miniconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading CLIP model...
Loaded CLIP model


/home/nightstalker/miniconda3/envs/pytorch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded CLIP processor
Evaluating CLIP model on validation set...


100%|██████████| 302/302 [00:47<00:00,  6.38it/s]


In [14]:
accuracy = accuracy_score(y_true_dh, y_pred_clip)
print(accuracy)

confusion = confusion_matrix(y_true_dh, y_pred_clip)
print(confusion)

report = classification_report(y_true_dh, y_pred_clip)
print(report)

0.6324503311258278
[[186 104]
 [  7   5]]
              precision    recall  f1-score   support

           0       0.96      0.64      0.77       290
           1       0.05      0.42      0.08        12

    accuracy                           0.63       302
   macro avg       0.50      0.53      0.43       302
weighted avg       0.93      0.63      0.74       302



## BLIP Benchmark

In [15]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

# Load BLIP model and processor (using the image captioning/VQA model)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

image_url = rows[0][4]

# Load your product image
image = Image.open(BytesIO(requests.get(image_url).content))

# Prepare your question prompt
question = "a real baby?"

# Process the inputs for BLIP (it will combine the image and text prompt)
inputs = processor(images=image, text=question, return_tensors="pt")

# Generate an answer (BLIP generates a text answer)
with torch.no_grad():
    out = model.generate(**inputs)

# Decode the generated text answer
answer = processor.decode(out[0], skip_special_tokens=True)
print("BLIP Answer:", answer)

/home/nightstalker/miniconda3/envs/pytorch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/nightstalker/miniconda3/envs/pytorch/lib/python3.11/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


BLIP Answer: a real baby? liquid


## Heuristic Benchmark

In [16]:
query_heuristic = """
SELECT *
FROM Baby_Bath_Skin_and_Grooming
WHERE name LIKE '%everyday%'
   OR sub_category LIKE '%everyday%'
   OR main_category LIKE '%everyday%'
   OR name LIKE '%regular%'
   OR name LIKE '%standard%'
   OR sub_category LIKE '%regular%'
   OR sub_category LIKE '%standard%';
"""

cursor.execute(query_heuristic)
rows_heuristic = cursor.fetchall()

In [17]:
rows_heuristic

[]